In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)


In [5]:
scales = [4]
selected_columns = ['x1', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    y = data[1]
    # FIXME: should be replaced by `s.where` when it's available. Now the result may be incorrect
    filter = (y < 20)
    # FIXME: not sure how to do filtering correctly here
    filtered_x = s.where(filter, x)
    filtered_y = s.where(filter, y)
    return s.linear_regression(filtered_x, filtered_y)


error = 0.05
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

result:  tensor([[[ 0.0224],
         [10.1720]]])


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:403: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y = torch.where(y==MagicNumber, torch.tensor(0.0), y)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:405: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if yo

In [7]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 4, param_scale: 4, scale_input_multiplier: 10) ------------->

+---------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error | max_error | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.0055365562 | -0.004655361 | 0         | -0.011954308 | 0.0055365562   | 0.004655361      | 0.011954308   | 0             | 0.000054859283     | -0.001968241       | 0.001968241            |
+---------------+--------------+-----------+--------------+----------------+------------------+----

result:  tensor([[[0.9953],
         [9.7380]]])
==== Generate & Calibrate Setting ====
scale:  [4]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":10,"lookup_range":[-400,512],"logrows":13,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":1518,"total_const_size":308,"model_instance_shapes":[[1],[1,2,1]],"model_output_scales":[0,4],"model_input_scales":[4,4],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709690487476}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.8701119422912598 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 1.0
witness result 2 : 9.75
==== Generating Proof ====
proof:  {'instances': [['78f113a919d1324cbee267b4320db42ee0170745a57013fa302f8139d7c36f18', '8dff683cfffbd97a4d94de7d897e03bb3f2fa18084beff4bbfbd152c2e2bcd16', '0100000000000000000000000000000000000000000000000000000000000000', '1000000000000000000000000000000000000000000000000000000000000000', '9c00000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x21ef95391b2d588bf775f55894b32a4d7edf4e217c2c651be6febf878c2c9e401d7adc17a42721973b6381a5531f475ef72c3102b6bb6e83c59b7ca22e8f2f682ddcc0a7d7399966703207ae49dbfcb08c2079fcbace9ab35d629ca47bc5585017d91e809e07a17aaa5683b5088bb15e179f985a07573e220a4c32559f1b6b881bc101e79216cda830c980a53ae6454f30c28e7350681a540b233ad4ab6a1a33083297b75bf0e375d76455c847afb6c3f5469fab912e2139b5f8b769cf9cd3822530ea793380a6c9667f3c3345e5b8da96d6c98329f1ca04

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [1.0, 9.75]


In [10]:
x1_arr=  np.array([
    7.1, 3.2, 8.6, 3.5, 0.1, 9.7, 2.3, 5.7, 2.8, 10.0, 6.0, 6.0, 9.1, 1.7, 9.2,
    0.2, 7.8, 3.7, 7.0, 2.5, 2.8, 5.9, 7.3, 2.9, 2.9, 3.5, 1.0, 9.7, 4.8, 0.9,
    7.1, 3.6, 8.2, 3.0, 7.6, 4.2, 5.2, 8.1, 6.3, 9.3, 8.8, 8.2, 6.7, 4.9, 5.4,
    9.8, 5.9, 7.1, 3.9, 9.3
  ])
y_arr =np.array([
    18.5, 5.5, 18.2, 9.0, 4.0, 19.5, 11.7, 17.9, 15.3, 20.8, 12.5, 21.5, 32.5,
    18.6, 23.9, 7.0, 16.9, 22.9, 31.0, 15.0, 8.5, 8.7, 28.9, 19.7, 12.5, 17.4,
    7.2, 25.5, 21.4, 15.7, 15.5, 8.2, 28.2, 19.5, 25.5, 12.5, 20.3, 21.7, 22.1,
    19.6, 32.2, 22.4, 20.6, 19.7, 20.8, 21.1, 21.8, 17.7, 21.1, 19.4
  ])
from scipy.stats import linregress
print(linregress(x1_arr, y_arr))
print(linregress(x1_arr[y_arr<20], y_arr[y_arr<20]))

LinregressResult(slope=1.5701100032164679, intercept=9.543682881955615, rvalue=0.6564886189558647, pvalue=2.2600615271814128e-07, stderr=0.2604036418835096, intercept_stderr=1.631311911135312)
LinregressResult(slope=0.9953446553446554, intercept=9.738045954045953, rvalue=0.5646151788621693, pvalue=0.0014190381578603627, stderr=0.2800138217099284, intercept_stderr=1.473954985506841)
